In [34]:
# implements SDP for approx. degradability bound https://github.com/vsiddhu/SDP-Quantum-OR/blob/main/Notebook%205%20-%20Quantum%20Channel%20Capacity.ipynb for Pauli channels

In [89]:
import numpy as np
import matplotlib.pyplot as plt
import HelperFunctionsforNotebook5 as crp
import picos as pic
import cvxopt as cvx
import qutip as qutip

In [90]:
print('Solvers available to picos on this machine :', pic.solvers.available_solvers())

Solvers available to picos on this machine : ['cvxopt']


In [91]:
def Pauli_Channel_Kraus(pX,pY,pZ):
    pI = 1 - pX - pY - pZ
    KI = np.sqrt(pI)*np.eye(2)
    KX = np.sqrt(pX)*np.array([[0,1],[1,0]])
    KY = np.sqrt(pY)*np.array([[0,-1j],[1j,0]])
    KZ = np.sqrt(pZ)*np.array([[1,0],[0,-1]])
    return [KI,KX,KY,KZ]
    

In [92]:
def hb(p):
    return -p*np.log2(p)-(1-p)*np.log2(1-p)

def entropy(p):
    e = 0
    for pi in p:
        if pi !=0:
            e =e-pi*np.log2(pi)
    return e

def coherent_info_Pauli(p):
    return 1-entropy(p)

In [93]:
def capacity_difference_bound(eps, dc):
    return eps*np.log2(dc-1)/2+eps*np.log2(dc)+hb(eps/2)+(1+eps/2)*hb(eps/(2+eps))
    

In [94]:
def approx_degradability_param_Pauli(pX,pY,pZ):
    krsLstF = Pauli_Channel_Kraus(pX,pY,pZ)
    (dc,db,da) = np.shape(krsLstF) #why dc? 
    #Choi-Jamiolkowski representations of channel and its complement 
    (jbaF, jcaF) = crp.krausToChoiJ(krsLstF)
    #Constants
    #----------
    JbaFPic = pic.Constant("J(B)_ba", jbaF)
    JcaFPic = pic.Constant("J(B)_ca", jcaF) #check reasoning!
    
    iMatA = pic.Constant('Ia', np.eye(da))
    iMatB = pic.Constant('Ib', np.eye(db))
    shpCA = (dc*da,dc*da)
    shpCB = (dc*db,dc*db)
    
    
    #Variables
    #----------
    ZPic = pic.HermitianVariable("Zca", shpCA)
    JPic = pic.HermitianVariable("Jcb", shpCB)
    mu = pic.RealVariable("mu")
    prob1P = pic.Problem()

    #Constraint
    #----------
    prob1P.add_constraint(ZPic >> 0)
    prob1P.add_constraint(JPic >> 0)
    
    ZaPic = pic.partial_trace(ZPic,subsystems=(0), dimensions=(dc,da))
    JbPic = pic.partial_trace(JPic,subsystems=(0), dimensions=(dc,db))
    
    prob1P.add_constraint(ZaPic << mu*iMatA)
    prob1P.add_constraint(JbPic == iMatB)
    
    JcaPic = crp.choiJOfChanInSeriesPic(JbaFPic,JPic,da,db,dc)
    prob1P.add_constraint(ZPic >> JcaFPic-JcaPic)
    

    #Objective
    #----------
    prob1P.set_objective('min',mu)
    
    #User readable view of the problem being composed in PICOS'
    #print(prob1P)
    #Solve the problem using cvxopt as a solver
    prob1P.solve(verbosity=False,solver='cvxopt')
    
    
    
    #Solver claims to have found optimal saolution
    mu1P =  prob1P.value
    eps1 = 2*mu1P
    return eps1, capacity_difference_bound(eps1, dc)+coherent_info_Pauli([1-pX-pY-pZ,pX,pY,pZ])


In [95]:
# Depolarizing Channel
upper_bound = []
hashing = []
p_range = np.arange(0.001,1, 0.01)
for p in p_range:
    pX = p/2
    pY = 0
    pZ = p/2
    e,c = approx_degradability_param_Pauli(pX,pY,pZ)
    print(e,c)
    upper_bound.append(c)
    hashing.append(1-entropy([1-pX-pY-pZ,pX,pY,pZ]))
    
plt.figure()
plt.grid()
plt.plot(p_range,upper_bound, label='upper_bound')
plt.plot(p_range,hashing, label='hashing')
plt.xlabel('p')
plt.legend()
plt.show()

4.005735442193845e-06 0.987685033833084
0.0004891726021144856 0.9095885963865206
0.0017989297206531436 0.8578024809801534
0.003952749373799927 0.8218655668199396
0.0069667053150079695 0.7985319830558134
0.01085307843448206 0.7860124056653829
0.015620036249648138 0.7830989702611424
0.021271410871221457 0.7888760056264776
0.02780658777140131 0.8025944959869357
0.035220511979352886 0.8236089736559967
0.04350381034048059 0.8513429496847553
0.05264302233612173 0.8852690515001647
0.06262092351887424 0.9248973584735263
0.07341692758750155 0.9697685846667711
0.0850075302764112 1.0194500695360043
0.09736679833264236 1.0735336178900454
0.11046685510337295 1.1316342510002289
0.12427836210943169 1.193389543558914
0.13877096934324593 1.2584591625657149
0.15391374246593786 1.3265245582883487
0.16967552728232282 1.397288482974158
0.1860252791131992 1.4704744988590643
0.20293233904161495 1.5458263399244256
0.2203666572550901 1.6231071280237268
0.23829898037967365 1.7020985448594088
0.256700988280887 1

KeyboardInterrupt: 

In [105]:
# Compute UE eq(5) in https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8046086
def isometric_ext(Kraus): 
    U = 0
    I = np.eye(len(Kraus))
    for j in range(len(Kraus)):
        jvector = I[:,j]
        jvector.shape = (len(Kraus),1)
        U = U + np.kron(Kraus[j],jvector)
    return U      

def UE(rho,Channel_Kraus,Complementary_Channel_Choi,dc,db):
    complementary_choi = qutip.Qobj(Complementary_Channel_Choi)
    complementary_choi.dims = [[dc,db],[dc,db]]
    print(complementary_choi)
    Complementary_Channel_Kraus = qutip.choi_to_kraus(complementary_choi)
    V = isometric_ext(Channel_Kraus)
    W = isometric_ext(Complementary_Channel_Kraus)
    dE = len(Channel_Kraus)
    dEt = np.shape(Complementary_Channel_Kraus[0])[1]
    dF = len(Complementary_Channel_Kraus)    
    omega_EEtF = np.kron(W,np.eye(dE))@V@rho@np.conj(np.transpose(np.kron(W,np.eye(dE))))
    omega_EtF = pic.partial_trace(omega_EEtF,subsystems=(0), dimensions=(dE,dEt,dF))
    omega_Et = pic.partial_trace(omega_EtF,subsystems=(1), dimensions=(dEt,dF))
    eigsEtF = np.linalg.eigvalsh(omega_EtF)
    eigsEt = np.linalg.eigvalsh(omega_Et)
    return entropy(eigsEtF)-entropy(eigsEt)
    

In [106]:
def objective(rho,pX,pY,pZ,dc,db): 
    krsLstF = Pauli_Channel_Kraus(pX,pY,pZ)
    #Choi-Jamiolkowski representations of channel and its complement 
    (jbaF, jcaF) = crp.krausToChoiJ(krsLstF)
    print(np.shape(jcaF))
    return UE(rho,krsLstF,jcaF,dc,db)


In [107]:
objective(np.eye(2)/2,0.1,0.2,0.1,4,2)

(8, 8)
Quantum object: dims = [[4, 2], [4, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 0.6       +0.j          0.        +0.j          0.        +0.j
   0.24494897+0.j          0.        +0.j          0.        +0.34641016j
   0.24494897+0.j          0.        +0.j        ]
 [ 0.        +0.j          0.6       +0.j          0.24494897+0.j
   0.        +0.j          0.        -0.34641016j  0.        +0.j
   0.        +0.j         -0.24494897+0.j        ]
 [ 0.        +0.j          0.24494897+0.j          0.1       +0.j
   0.        +0.j          0.        -0.14142136j  0.        +0.j
   0.        +0.j         -0.1       +0.j        ]
 [ 0.24494897+0.j          0.        +0.j          0.        +0.j
   0.1       +0.j          0.        +0.j          0.        +0.14142136j
   0.1       +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.34641016j  0.        +0.14142136j
   0.        +0.j          0.2       +0.j          0.        +0.j
   0.    

TypeError: arrays to stack must be passed as a "sequence" type such as list or tuple.

In [ ]:
def constraint():
    #ωE ˜E F = (W ⊗ 1)VρV †(W ⊗ 1)†